In [1]:
import logging
import time
from pathlib import Path
from crewai.tools import BaseTool
from crewai_tools import DirectoryReadTool,FileReadTool

from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

_log = logging.getLogger(__name__)


class DoclingPDFParserTool(BaseTool):
    """
    Parse a PDF using Docling with full layout, equation, and image preservation.
    Exports:
    - Page images
    - Figure images
    - Table images
    - Markdown (embedded + referenced images)
    - HTML (referenced images)
    """

    name: str = "docling_pdf_parser"
    description: str = (
        "Takes PDF path as input and Parses a PDF using Docling with formula enrichment and high-resolution "
        "image extraction. Exports page images, figure images, table images, "
        "Markdown (embedded and referenced), and HTML."
    )

    def _run(
        self,
        pdf_path: str,
        output_dir: str = "parsed_output",
        image_resolution_scale: float = 2.0,
        embed_images_in_markdown: bool = False,
    ) -> str:
        start_time = time.time()

        pdf_path = Path(pdf_path)
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)

        # -------------------------
        # Configure Docling pipeline
        # -------------------------
        pipeline_options = PdfPipelineOptions(
            do_formula_enrichment=True
        )
        pipeline_options.images_scale = image_resolution_scale
        pipeline_options.generate_page_images = True
        pipeline_options.generate_picture_images = True

        doc_converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_options=pipeline_options
                )
            }
        )

        # -------------------------
        # Convert document
        # -------------------------
        conv_res = doc_converter.convert(str(pdf_path))
        document = conv_res.document
        doc_stem = pdf_path.stem

        # -------------------------
        # Save page images
        # -------------------------
        page_image_count = 0
        for page in document.pages.values():
            page_image_count += 1
            page_image_path = output_dir / f"{doc_stem}-page-{page.page_no}.png"
            with page_image_path.open("wb") as fp:
                page.image.pil_image.save(fp, format="PNG")

        # -------------------------
        # Save figure & table images
        # -------------------------
        table_counter = 0
        picture_counter = 0

        for element, _level in document.iterate_items():
            if isinstance(element, TableItem):
                table_counter += 1
                table_path = output_dir / f"{doc_stem}-table-{table_counter}.png"
                with table_path.open("wb") as fp:
                    element.get_image(document).save(fp, "PNG")

            elif isinstance(element, PictureItem):
                picture_counter += 1
                picture_path = output_dir / f"{doc_stem}-figure-{picture_counter}.png"
                with picture_path.open("wb") as fp:
                    element.get_image(document).save(fp, "PNG")

        # -------------------------
        # Save Markdown
        # -------------------------
        if embed_images_in_markdown:
            md_path = output_dir / f"{doc_stem}-embedded.md"
            document.save_as_markdown(md_path, image_mode=ImageRefMode.EMBEDDED)
        else:
            md_path = output_dir / f"{doc_stem}-refs.md"
            document.save_as_markdown(md_path, image_mode=ImageRefMode.REFERENCED)

        # elapsed = time.time() - start_time

        return {
            "paper_markdown": str(md_path),
            "figures_dir": str(output_dir),
  
    }


In [3]:
result = generate_explainer("/Users/abhyuday/Downloads/2406.08085.pdf")

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: b7a531e7-8dc9-40cb-b044-a733e8b4f4df                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 7eb30053-7f49-4f92-bd06-3767a68c1b6e
    Status: Executing Task...
╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Crew Ma

12:09:47 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= qwen2.5vl:7b; provider = ollama
2025-12-23 12:09:47,236 - INFO - 
LiteLLM completion() model= qwen2.5vl:7b; provider = ollama
12:10:03 - LiteLLM:INFO: utils.py:1311 - Wrapper: Completed Call, calling success_handler
2025-12-23 12:10:03,297 - INFO - Wrapper: Completed Call, calling success_handler


🚀 Crew: crew
└── 📋 Task: 7eb30053-7f49-4f92-bd06-3767a68c1b6e
    Status: Executing Task...
/Users/abhyuday/envs/shodh/lib/python3.13/site-packages/pydantic/main.py:463: 
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected 
`Message` - serialized value may not be as expected 
[input_value=Message(content='Thought:... reasoning_content=None), 
input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized 
value may not be as expected 
[input_value=Choices(finish_reason='st...reasoning_content=None)), 
input_type=Choices])
  return self.__pydantic_serializer__.to_python(
🚀 Crew: crew
└── 📋 Task: 7eb30053-7f49-4f92-bd06-3767a68c1b6e
    Status: Executing Task...
    └── 🔧 Using docling_pdf_parser (1)

2025-12-23 12:10:03,431 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-23 12:10:03,483 - INFO - Going to convert document batch...
2025-12-23 12:10:03,483 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 0c10c4ac0801dd915a35ad87dac96c89
2025-12-23 12:10:03,504 - INFO - Loading plugin 'docling_defaults'
2025-12-23 12:10:03,506 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-23 12:10:03,513 - INFO - Loading plugin 'docling_defaults'
2025-12-23 12:10:03,517 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']


🚀 Crew: crew
└── 📋 Task: 7eb30053-7f49-4f92-bd06-3767a68c1b6e
    Status: Executing Task...
🚀 Crew: crew
└── 📋 Task: 7eb30053-7f49-4f92-bd06-3767a68c1b6e
    Status: Executing Task...
🚀 Crew: crew
└── 📋 Task: 7eb30053-7f49-4f92-bd06-3767a68c1b6e
    Status: Executing Task...
    └── 🔧 Using docling_pdf_parser (1)

...

12:10:54 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= qwen2.5vl:7b; provider = ollama


🚀 Crew: crew
├── 📋 Task: 7eb30053-7f49-4f92-bd06-3767a68c1b6e
│   Assigned to: Crew Manager
│   Status: ✅ Completed
│   ├── 🔧 Used docling_pdf_parser (1)
│   └── 🧠 Thinking...
└── 📋 Task: 734ad8e9-db32-42a1-91ed-b53189d86f6a
    Status: Executing Task...
    ├── 🔧 Used List files in directory (1)
    └── 🔧 Using Read a file's content (1)


2025-12-23 12:10:54,559 - INFO - 
LiteLLM completion() model= qwen2.5vl:7b; provider = ollama


╭────────────────────────── 🔧 Agent Tool Execution ───────────────────────────╮
│                                                                              │
│  Agent: Crew Manager                                                         │
│                                                                              │
│  Thought: Thought: I need to read the content of the markdown file to        │
│  understand the paper's context and objectives.                              │
│                                                                              │
│  Using Tool: Read a file's content                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── Tool Input ─────────────────────────────────╮
│                                                                              │
│  {                        

APIConnectionError: litellm.APIConnectionError: OllamaException - {"error":"model runner has unexpectedly stopped, this may be due to resource limitations or an internal error, check ollama server logs for details"}